## Giới thiệu và Nhập Thư viện

- **numpy**: Thư viện để làm việc với mảng và tính toán số học.
- **random**: Dùng để tạo số ngẫu nhiên (không sử dụng trực tiếp trong code này).
- **json**: Dùng để đọc file JSON chứa dữ liệu intents.
- **torch**: Thư viện PyTorch để xây dựng và huấn luyện mô hình học sâu.
- **torch.nn**: Chứa các thành phần mạng nơ-ron như lớp, hàm mất mát.
- **torch.utils.data**: Chứa các công cụ để xử lý dữ liệu như Dataset, DataLoader.
- **nltk_utils**: Module tùy chỉnh chứa các hàm xử lý văn bản như tokenize, stem, bag_of_words.
- **model**: Module chứa lớp NeuralNet định nghĩa kiến trúc mạng nơ-ron.

In [7]:
import numpy as np
import random
import json

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from nltk_utils import bag_of_words, tokenize, stem
from model import NeuralNet

## Tải và Xử lý Sơ bộ Dữ liệu Ý định (Intents)

- Đọc file `intents.json` để lấy danh sách các ý định.
- Tạo danh sách:
  - `all_words`: Lưu tất cả các từ từ patterns.
  - `tags`: Lưu các nhãn ý định.
  - `xy`: Lưu cặp (câu tokenized, tag).
  - `all_sentences`: Lưu tất cả các câu tokenized để tính IDF.

In [8]:
with open('intents.json', encoding='utf-8') as f:
    intents = json.load(f)

all_words = []
tags = []
xy = []
all_sentences = []
for intent in intents['intents']:
    tag = intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        w = tokenize(pattern)
        all_words.extend(w)
        xy.append((w, tag))
        all_sentences.append(w)

## Tiền xử lý Dữ liệu Văn bản

- Loại bỏ các ký tự không cần thiết (`ignore_words`).
- Stem và chuyển các từ thành chữ thường.
- Loại bỏ từ trùng lặp và sắp xếp `all_words` và `tags`.

In [9]:
ignore_words = ['?', '.', '!']
all_words = [stem(w) for w in all_words if w not in ignore_words]
all_words = sorted(set(all_words))
tags = sorted(set(tags))

print(len(xy), "patterns")
print(len(tags), "tags:", tags)
print(len(all_words), "unique stemmed words:", all_words)

66 patterns
12 tags: ['delivery', 'feedback', 'funny', 'goodbye', 'greeting', 'hours', 'items', 'location', 'payments', 'reservation', 'special_request', 'thanks']
196 unique stemmed words: [',', '5', 'ai', 'bao', 'biết', 'biệt', 'buổi', 'bàn', 'bán', 'bâi', 'bạn', 'bản', 'bật', 'bằng', 'bị', 'cay', 'chay', 'chi', 'cho', 'chuyện', 'chuẩn', 'chào', 'chúc', 'chấp', 'chỉ', 'cuối', 'các', 'cách', 'câu', 'có', 'cười', 'cảm', 'cần', 'cầu', 'cọc', 'của', 'cửa', 'dị', 'dịch', 'dụng', 'gian', 'giao', 'giá', 'giúp', 'giới', 'giờ', 'gì', 'góp', 'gần', 'gặp', 'gửi', 'hay', 'hoặc', 'hài', 'hàng', 'hành', 'hãi', 'hôm', 'hơn', 'hước', 'hẹn', 'hồi', 'hữu', 'khen', 'khi', 'khác', 'khách', 'không', 'khỏe', 'kiến', 'kể', 'liên', 'là', 'làm', 'lâu', 'lòng', 'lưu', 'lại', 'lần', 'lễ', 'mastercard', 'menu', 'momo', 'muối', 'muốn', 'món', 'mấi', 'mất', 'mặt', 'một', 'mới', 'mở', 'nay', 'nghe', 'ngài', 'người', 'ngợi', 'nhanh', 'nhiều', 'nhà', 'nhánh', 'nhé', 'nhóm', 'nhất', 'nhận', 'nhật', 'những', 'nài', 'n

## Tạo Dữ liệu Huấn luyện

- Tạo `X_train` (vector TF-IDF) và `y_train` (nhãn).
- Sử dụng hàm `bag_of_words` với `all_sentences` để tính TF-IDF.

In [10]:
X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    bag = bag_of_words(pattern_sentence, all_words, all_sentences)
    X_train.append(bag)
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

## Định nghĩa Siêu tham số và Kích thước Mô hình

- Định nghĩa các siêu tham số: số epoch, kích thước batch, tốc độ học, v.v.
- Xác định kích thước đầu vào và đầu ra dựa trên dữ liệu.

In [17]:
num_epochs = 800
batch_size = 8
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)
print("input_size:", input_size)
print("output_size:", output_size)

input_size: 196
output_size: 12


## Tạo Lớp ChatDataset

- Tạo lớp `ChatDataset` để quản lý dữ liệu huấn luyện.
- Hỗ trợ lập chỉ mục và trả về kích thước dataset.

In [18]:
class ChatDataset(Dataset):

    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.n_samples

## Tạo DataLoader

- Tạo `DataLoader` để chia dữ liệu thành các batch và xáo trộn.

In [19]:
dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

## Khởi tạo Mô hình, Hàm Mất mát và Trình Tối ưu hóa

- Khởi tạo mô hình `NeuralNet` và chuyển sang thiết bị (`cuda` hoặc `cpu`).
- Sử dụng `CrossEntropyLoss` và tối ưu hóa `Adam`.

In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = NeuralNet(input_size, hidden_size, output_size).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

## Huấn luyện Mô hình

- Lặp qua các epoch, thực hiện forward pass, backward pass, và cập nhật tham số.
- In giá trị hàm mất mát sau mỗi 100 epoch.

In [21]:
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)
        
        outputs = model(words)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [100/800], Loss: 0.5040
Epoch [200/800], Loss: 0.2754
Epoch [300/800], Loss: 0.0208
Epoch [400/800], Loss: 0.0066
Epoch [500/800], Loss: 0.0041
Epoch [600/800], Loss: 0.0033
Epoch [700/800], Loss: 0.0008
Epoch [800/800], Loss: 0.0006


## In Mất mát Cuối cùng và Lưu Mô hình

- In giá trị hàm mất mát cuối cùng.
- Lưu trạng thái mô hình và thông tin vào file `data.pth`.

In [22]:
print(f'final loss: {loss.item():.4f}')

data = {
"model_state": model.state_dict(),
"input_size": input_size,
"hidden_size": hidden_size,
"output_size": output_size,
"all_words": all_words,
"tags": tags
}

FILE = "data.pth"
torch.save(data, FILE)

print(f'training complete. file saved to {FILE}')

final loss: 0.0006
training complete. file saved to data.pth
